<a href="https://colab.research.google.com/github/vamshap/PySpark-Challenges/blob/main/TasksStatus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,window,row_number,date_add,min,max
from pyspark.sql.types import StructType, StructField, StringType, DateType
from pyspark.sql.window import Window
from datetime import datetime


#write a code to print the Max and Min date of a continous status of Success , Fail example 2019-01-01  "2019-01-03" "Success" and
# "2019-01-04" "2019-01-05" "Fail"

# Initialize Spark Session
spark = SparkSession.builder.appName("CreateTasksTable").getOrCreate()

# Define the schema for the DataFrame
schema = StructType([
    StructField("date_value", DateType(), True),
    StructField("state", StringType(), True)
])

# Data to insert into the DataFrame
data = [
    (datetime.strptime("2019-01-01", "%Y-%m-%d").date(), "success"),
    (datetime.strptime("2019-01-02", "%Y-%m-%d").date(), "success"),
    (datetime.strptime("2019-01-03", "%Y-%m-%d").date(), "success"),
    (datetime.strptime("2019-01-04", "%Y-%m-%d").date(), "fail"),
    (datetime.strptime("2019-01-05", "%Y-%m-%d").date(), "fail"),
    (datetime.strptime("2019-01-06", "%Y-%m-%d").date(), "success")
]

# Create the DataFrame
tasks_df = spark.createDataFrame(data, schema=schema)

tasks_df.show()
Window_Spec = Window.partitionBy("state").orderBy("date_value")

newcol = tasks_df.withColumn("Grop",date_add(col("date_value"),-1*row_number().over(Window_Spec)))
newcol.groupBy("state","Grop").agg(min("date_value").alias("min_date"),max("date_value").alias("max_date")).select("state","min_date","max_date").show()











+----------+-------+
|date_value|  state|
+----------+-------+
|2019-01-01|success|
|2019-01-02|success|
|2019-01-03|success|
|2019-01-04|   fail|
|2019-01-05|   fail|
|2019-01-06|success|
+----------+-------+

+-------+----------+----------+
|  state|  min_date|  max_date|
+-------+----------+----------+
|   fail|2019-01-04|2019-01-05|
|success|2019-01-01|2019-01-03|
|success|2019-01-06|2019-01-06|
+-------+----------+----------+

